In [ ]:
library(ggplot2)
library(reshape2)
library(data.table)
library(knitr)
library(stringr)
library(NMF)
library(rsvd)
library(lme4)
library(RColorBrewer)
library(Seurat)
library(SingleCellExperiment)
library(data.table)
library(scater)
datasets <- readRDS(".../RData/SEZ_seurat_object.rds")


In [ ]:
setwd(".../Test_Code")

In [ ]:
DefaultAssay(datasets) <- "RNA"
levels(datasets)

In [ ]:
colnames(datasets@meta.data)

In [ ]:
clusters <- levels(datasets)


for (cluster in clusters){
    
    print(cluster)

    temp <- subset(datasets, ident = as.character(cluster)) #choose the cluster

    counts <- table( temp$case,temp@meta.data$cluster) #  cell fractions per donor
    counts<-data.table(counts)
    counts<-subset(counts, V2==cluster)
    donors_min_cell<-subset(counts,N>2)
    donors_min_cell_2<-donors_min_cell$V1
    write.csv(donors_min_cell_2,paste0('MAST_025_01/', cluster, '-MAST_donors.csv'))
    
    print(donors_min_cell_2)
    
    temp <- SetIdent(temp, value = "case")
    temp <- subset(temp, ident = donors_min_cell_2) 
    
    DefaultAssay(temp) <- 'RNA'         ##Calculate genes FC between the two groups          
    fc.results<-FoldChange(temp,  group.by="group", ident.1="Y", ident.2 = "M", # 
                     slot = 'data', pseudocount.use = 1) #choose the groups A and B, pulls from the slot 'data' and add a 1 to lognorm data
    print('dimensions before filter:')
    print(dim(temp))
    
    #feature selection based on min percentage
    min.pct=0.25  #put a value only test genes that are detected in a minimum fraction of min.pct cells in either of the two populations #maybe choose it with the thresholdSCRNACountMatrix() from MAST suggestion?
    min.diff.pct = -Inf  #  minimum difference in expression between the two groups, set to -Inf for our propose
    logfc.threshold=0.25# genes need at least X-fold difference (log-scale) between the two groups of cells. Default is 0.25 Increasing logfc.
    
    alpha.min <- pmax(fc.results$pct.1, fc.results$pct.2)
    names(x = alpha.min) <- rownames(x = fc.results)
    features <- names(x = which(x = alpha.min > min.pct))
    alpha.diff <- alpha.min - pmin(fc.results$pct.1, fc.results$pct.2)
    features <- names(x = which(x = alpha.min > min.pct & alpha.diff > min.diff.pct))
    print(length(features))

    #feature selection based on logFC
    total.diff <- fc.results[, 1] #first column is logFC
    names(total.diff) <- rownames(fc.results)
    features.diff <- names(x = which(x = abs(x = total.diff) > logfc.threshold))
    print(length(features.diff))
    features <- intersect(x = features, y = features.diff)
    print(length(features))

    #subset for the selected features before creating the single cell experiment
    seu_subset<-subset(x = temp, features = features)
    DefaultAssay(seu_subset) <- "RNA"
    print('dimensions after prefilter:')
    print(dim(seu_subset))
    
    # convert to single cell assay 
    gc()
    logcounts <- as.matrix(GetAssayData(seu_subset, assay = "RNA", slot = 'data'))
    meta <- as.data.frame(seu_subset@meta.data)
    fdata <- data.frame(primerid=rownames(seu_subset), stringsAsFactors = F)
    
    print('making MAST object..')
    sca <- MAST::FromMatrix(exprsArray = list(logNorm = logcounts), cData = meta, fData = fdata)
    rm(logcounts, meta, fdata)

    print('saving MAST object from..')

    saveRDS(sca, file = paste0('MAST_025_01/', cluster, '-sca-filt.rds'))
    #rm(sca)
    }

In [ ]:
clusters<-levels(datasets)
for (cluster in clusters){
    print(cluster)
    sca<-readRDS(file=paste0("MAST_025_01/",paste0( cluster, "-sca-filt.rds")))
    
    # 1. prepare the sca
    cond<-factor(colData(sca)$group)
    cond<-relevel(cond,"0")
    colData(sca)$group<-cond
    cond_id<-factor(colData(sca)$case)
    colData(sca)$case<-cond_id 
    

    cdr2 <-colSums(assay(sca)>0)
    colData(sca)$cngeneson <- scale(cdr2)
    FCTHRESHOLD <- 0.1#log2(1.19) ### i lowered it!
    
    ## 2. fit the model
    zlmCond <-MAST::zlm(~ group + cngeneson + (1|case), sca, exprs_value='logNorm', method = 'glmer', ebayes = FALSE) # also tried strict.convergence TRUE and FALSE
# BrainPH ++ cngeneson
    # create contrast
    summaryCond <- summary(zlmCond, doLRT="groupM") #p valores ajustados con significancia
    summaryDt <- summaryCond$datatable
    fcHurdle <- merge(summaryDt[contrast=='groupM' & component=='H',.(primerid, `Pr(>Chisq)`)],
    #hurdle P values
    summaryDt[contrast=='groupM' & component=='logFC', .(primerid, coef, ci.hi,
    ci.lo)], by='primerid') #logFC coefficients

    ## 3. investigate output
    summaryDt <- summaryCond$datatable
    fcHurdle <- merge(summaryDt[contrast=='groupM' & component=='H',.(primerid, `Pr(>Chisq)`)], #hurdle P values
                      summaryDt[contrast=='groupM' & component=='logFC', .(primerid, coef, ci.hi, ci.lo)], by='primerid') #logFC coefficients
    fcHurdle[,fdr:=p.adjust(`Pr(>Chisq)`, 'fdr')]
    fcHurdleSig <- merge(fcHurdle[fdr<.05 & abs(coef)>FCTHRESHOLD], as.data.table(mcols(sca)), by='primerid')
    setorder(fcHurdleSig, fdr)
    setorder(fcHurdle, fdr)
    # print some checks
    print(paste('we modelled', nrow(sca), 'genes for', cluster))
    print('number of converged models for continuous component:')
    print(sum(zlmCond@converged[,1] == "TRUE")) # continuous component 

    print('number of converged models for discrete component:')
    print(sum(zlmCond@converged[,2] == "TRUE")) # discrete model 
    
    # Write CSV out in a fast way
    fwrite(fcHurdle, paste0("MAST_025_01/", cluster,"-MAST-mixed.csv"))
    fwrite(fcHurdleSig, paste0("MAST_025_01/results/", cluster,"MAST-mixed-sign.csv"))

    a<-c(dim(fcHurdleSig))
    
    str(zlmCond@LMlike@fitC)
    print(paste('number of DE genes for', cluster, ': ', a[1]))
}